In [15]:
from pyspark.sql.types import IntegerType

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

In [3]:
spark.sql("SELECT 'hello' as my_col").show()

+------+
|my_col|
+------+
| hello|
+------+



In [86]:
# df = spark.read.format("csv").option("header", "true").load("Pokemon.csv")
df = spark.read.csv("Pokemon.csv", header = True)

In [88]:
# Name of pokemon that has better attack
df = df.withColumn("Attack", df["Attack"].cast(IntegerType()))

In [89]:
sel = df.select(df["Type 1"], df["Name"], df["Attack"].astype(IntegerType()))

In [90]:
sel.orderBy(sel["Attack"].desc()).limit(5).show()

+-------+--------------------+------+
| Type 1|                Name|Attack|
+-------+--------------------+------+
|Psychic| MewtwoMega Mewtwo X|   190|
|    Bug|HeracrossMega Her...|   185|
|Psychic|  DeoxysAttack Forme|   180|
| Dragon|RayquazaMega Rayq...|   180|
| Ground|GroudonPrimal Gro...|   180|
+-------+--------------------+------+



In [53]:
top_5 = df.orderBy("Attack", ascending = False).limit(5) 
# NOTE: if you put .show() at the end of the line above, you will see the data frame 
# but when you assign it to top_5 you're assigning the show which is temporary
# To assign, you need to remove the .show()

top_5.show()

+---+--------------------+-------+--------+-----+---+------+-------+-------+-------+-----+----------+---------+
|  #|                Name| Type 1|  Type 2|Total| HP|Attack|Defense|Sp. Atk|Sp. Def|Speed|Generation|Legendary|
+---+--------------------+-------+--------+-----+---+------+-------+-------+-------+-----+----------+---------+
|150| MewtwoMega Mewtwo X|Psychic|Fighting|  780|106|   190|    100|    154|    100|  130|         1|     True|
|214|HeracrossMega Her...|    Bug|Fighting|  600| 80|   185|    115|     40|    105|   75|         2|    False|
|386|  DeoxysAttack Forme|Psychic|    null|  600| 50|   180|     20|    180|     20|  150|         3|     True|
|384|RayquazaMega Rayq...| Dragon|  Flying|  780|105|   180|    100|    180|    100|  115|         3|     True|
|383|GroudonPrimal Gro...| Ground|    Fire|  770|100|   180|    160|    150|     90|   90|         3|     True|
+---+--------------------+-------+--------+-----+---+------+-------+-------+-------+-----+----------+---

In [25]:
df.groupBy("Type 1").count().orderBy("count", ascending = False).show()

+--------+-----+
|  Type 1|count|
+--------+-----+
|   Water|  112|
|  Normal|   98|
|   Grass|   70|
|     Bug|   69|
| Psychic|   57|
|    Fire|   52|
|    Rock|   44|
|Electric|   44|
|   Ghost|   32|
|  Dragon|   32|
|  Ground|   32|
|    Dark|   31|
|  Poison|   28|
|Fighting|   27|
|   Steel|   27|
|     Ice|   24|
|   Fairy|   17|
|  Flying|    4|
+--------+-----+



In [29]:
df.select("Name").limit(5).show()

+--------------------+
|                Name|
+--------------------+
|           Bulbasaur|
|             Ivysaur|
|            Venusaur|
|VenusaurMega Venu...|
|          Charmander|
+--------------------+



In [35]:
df.select((df['Attack'] > 99).alias("Big Attack")).groupBy("Big Attack").count().show()

+----------+-----+
|Big Attack|count|
+----------+-----+
|      true|  210|
|     false|  590|
+----------+-----+



In [36]:
df.describe().show()

+-------+------------------+----------------+------+------+------------------+------------------+-----------------+------------------+----------------+-----------------+------------------+------------------+---------+
|summary|                 #|            Name|Type 1|Type 2|             Total|                HP|           Attack|           Defense|         Sp. Atk|          Sp. Def|             Speed|        Generation|Legendary|
+-------+------------------+----------------+------+------+------------------+------------------+-----------------+------------------+----------------+-----------------+------------------+------------------+---------+
|  count|               800|             800|   800|   414|               800|               800|              800|               800|             800|              800|               800|               800|      800|
|   mean|         362.81375|            null|  null|  null|          435.1025|          69.25875|         79.00125|           73

In [52]:
top_5.write.csv("top_5_pokemon.csv")

In [50]:
top_5.toPandas().to_csv('top_5_pokemon_file.csv')

In [56]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(df['Type 1']).orderBy(df['Attack'].desc())

In [68]:
top_5_type = df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 5).\
    select("Type 1", "Name", "Attack", "rank")
top_5_type.show(100, truncate = False) 

+--------+-------------------------+------+----+
|Type 1  |Name                     |Attack|rank|
+--------+-------------------------+------+----+
|Water   |GyaradosMega Gyarados    |155   |1   |
|Water   |SwampertMega Swampert    |150   |2   |
|Water   |KyogrePrimal Kyogre      |150   |2   |
|Water   |SharpedoMega Sharpedo    |140   |4   |
|Water   |Kingler                  |130   |5   |
|Poison  |Toxicroak                |106   |1   |
|Poison  |Muk                      |105   |2   |
|Poison  |Nidoking                 |102   |3   |
|Poison  |Seviper                  |100   |4   |
|Poison  |Garbodor                 |95    |5   |
|Steel   |AegislashBlade Forme     |150   |1   |
|Steel   |MetagrossMega Metagross  |145   |2   |
|Steel   |AggronMega Aggron        |140   |3   |
|Steel   |Metagross                |135   |4   |
|Steel   |SteelixMega Steelix      |125   |5   |
|Rock    |Rampardos                |165   |1   |
|Rock    |TyranitarMega Tyranitar  |164   |2   |
|Rock    |DiancieMeg

In [69]:
top_5_type.toPandas().to_csv('top_5_type_pokemon_file.csv')

In [70]:
df.printSchema()

root
 |-- #: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type 1: string (nullable = true)
 |-- Type 2: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- HP: string (nullable = true)
 |-- Attack: integer (nullable = true)
 |-- Defense: string (nullable = true)
 |-- Sp. Atk: string (nullable = true)
 |-- Sp. Def: string (nullable = true)
 |-- Speed: string (nullable = true)
 |-- Generation: string (nullable = true)
 |-- Legendary: string (nullable = true)



In [79]:
# Seems like another possible solution rather than use the window function is to make a temp table

from pyspark.sql.functions import col

table = df.select(col("Name").alias("name"), col("Attack").alias("attack"), col("Type 1").alias("type"))
table.registerTempTable("table")
table.show()

+--------------------+------+-----+
|                name|attack| type|
+--------------------+------+-----+
|           Bulbasaur|    49|Grass|
|             Ivysaur|    62|Grass|
|            Venusaur|    82|Grass|
|VenusaurMega Venu...|   100|Grass|
|          Charmander|    52| Fire|
|          Charmeleon|    64| Fire|
|           Charizard|    84| Fire|
|CharizardMega Cha...|   130| Fire|
|CharizardMega Cha...|   104| Fire|
|            Squirtle|    48|Water|
|           Wartortle|    63|Water|
|           Blastoise|    83|Water|
|BlastoiseMega Bla...|   103|Water|
|            Caterpie|    30|  Bug|
|             Metapod|    20|  Bug|
|          Butterfree|    45|  Bug|
|              Weedle|    35|  Bug|
|              Kakuna|    25|  Bug|
|            Beedrill|    90|  Bug|
|BeedrillMega Beed...|   150|  Bug|
+--------------------+------+-----+
only showing top 20 rows



In [83]:
top_5_type_temp_table = spark.sql('with my_ranks as (\
           select *, rank() over (partition by type order by attack desc) as rank from table)\
           select type, name, rank from my_ranks\
           where rank <= 5')
top_5_type_temp_table.show()

+------+--------------------+----+
|  type|                name|rank|
+------+--------------------+----+
| Water|GyaradosMega Gyar...|   1|
| Water|SwampertMega Swam...|   2|
| Water| KyogrePrimal Kyogre|   2|
| Water|SharpedoMega Shar...|   4|
| Water|             Kingler|   5|
|Poison|           Toxicroak|   1|
|Poison|                 Muk|   2|
|Poison|            Nidoking|   3|
|Poison|             Seviper|   4|
|Poison|            Garbodor|   5|
| Steel|AegislashBlade Forme|   1|
| Steel|MetagrossMega Met...|   2|
| Steel|   AggronMega Aggron|   3|
| Steel|           Metagross|   4|
| Steel| SteelixMega Steelix|   5|
|  Rock|           Rampardos|   1|
|  Rock|TyranitarMega Tyr...|   2|
|  Rock| DiancieMega Diancie|   3|
|  Rock|            Archeops|   4|
|  Rock|AerodactylMega Ae...|   5|
+------+--------------------+----+
only showing top 20 rows

